In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import scattertext as st
import re, io
from pprint import pprint
import numpy as np
from scipy.stats import rankdata, hmean, norm
import spacy
from spacy.lemmatizer import Lemmatizer
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import gensim.corpora as corpora
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import os, pkgutil, json, urllib
from urllib.request import urlopen
from gensim.models import Word2Vec
from IPython.display import IFrame
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
from gensim.models import Phrases
import codecs
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn.decomposition import PCA
from gensim.models.word2vec import LineSentence
import gensim
display(HTML("<style>.container { width:98% !important; }</style>"))

/Users/bassim/anaconda3/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [4]:
data = pd.read_csv('reviews.csv')
data = data[['reviews','stars']]
data['sentiment'] = ['positive' if x in [4,5] else 'negative' for x in data['stars']]

In [5]:
nlp = spacy.load('en')

In [6]:
data['first_process'] = [gensim.utils.simple_preprocess(str(x), deacc=True) for x in data['reviews']] 

In [7]:
sentences = []
for sentence in data['first_process']:
    sample = []
    for word in sentence:
        if word not in stop_words:
            sample.append(wordnet_lemmatizer.lemmatize(word))
    sentences.append(sample)


In [8]:
bigram = gensim.models.Phrases([x for x in sentences], min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[sentences], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [9]:
data_lemmatized = [trigram_mod[bigram_mod[sentences[x]]] for x in range(len(sentences))]

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [11]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [12]:
# Build LDA model. takes a little time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=200,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.409736  0.075831       1        1  63.224541
0     -0.048646  0.114811       2        1   9.744812
4     -0.035424 -0.390814       3        1   9.246040
1      0.079960  0.078620       4        1   6.463660
2      0.133791  0.067246       5        1   4.677201
3      0.145051  0.019551       6        1   3.880923
6      0.135004  0.034756       7        1   2.762821, topic_info=     Category         Freq                  Term        Total  loglift  \
term                                                                     
451   Default  1357.000000                  lens  1357.000000  30.0000   
14    Default  1319.000000                camera  1319.000000  29.0000   
1203  Default   806.000000               helpful   806.000000  28.0000   
57    Default  4303.000000                   wex  4303.000000  27.0000   
79    Default  3812.000000               service  3812.000000  26.0000   
387   Default   711.000000                   new   711.000000  25.0000   
12    Default   627.000000                bought   627.000000  24.0000   
656   Default   519.000000                 staff   519.000000  23.0000   
60    Default   513.000000                  fast   513.000000  22.0000   
20    Default  2658.000000              delivery  2658.000000  21.0000   
3     Default   415.000000                advice   415.000000  20.0000   
719   Default   464.000000                 canon   464.000000  19.0000   
299   Default   436.000000                   got   436.000000  18.0000   
170   Default  2229.000000                   day  2229.000000  17.0000   
285   Default   561.000000                 could   561.000000  16.0000   
236   Default   348.000000             efficient   348.000000  15.0000   
296   Default   339.000000          professional   339.000000  14.0000   
237   Default   334.000000              friendly   334.000000  13.0000   
63    Default  1979.000000                  good  1979.000000  12.0000   
38    Default  1922.000000                 order  1922.000000  11.0000   
396   Default  1070.000000                  used  1070.000000  10.0000   
483   Default  1779.000000                 price  1779.000000   9.0000   
314   Default   331.000000                  call   331.000000   8.0000   
8     Default   328.000000                  back   328.000000   7.0000   
98    Default  1696.000000                always  1696.000000   6.0000   
666   Default   237.000000                 issue   237.000000   5.0000   
1046  Default   251.000000                   guy   251.000000   4.0000   
51    Default  1563.000000                  time  1563.000000   3.0000   
3403  Default   222.000000                highly   222.000000   2.0000   
1696  Default   277.000000             condition   277.000000   1.0000   
...       ...          ...                   ...          ...      ...   
2335   Topic7    38.949146               printer    39.549557   3.5736   
2736   Topic7    38.591908             operating    39.201225   3.5733   
3669   Topic7    35.836308  competitively_priced    36.437447   3.5723   
3650   Topic7    35.395813              schedule    36.064217   3.5702   
6414   Topic7    35.133331               fastest    35.807888   3.5699   
359    Topic7    30.560219                 light    31.160999   3.5694   
5115   Topic7    34.722939             oversight    35.416424   3.5691   
807    Topic7    28.877790               handled    29.479412   3.5683   
1840   Topic7    27.920784              everyone    28.521975   3.5676   
3504   Topic7    27.106045                    ii    27.710278   3.5669   
1704   Topic7    27.029173              exchange    27.632421   3.5668   
139    Topic7    25.945847                sensor    26.546770   3.5660   
193    Topic7    25.527637                 shoot    26.129631   3.5656   
5618   Topic7    33.949390                aspire    34.782677   3.5